# Lab 12: Image Calibrations
Based on the astropy tutorial http://docs.astropy.org/en/stable/io/fits/

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.cmap'] = 'Greys_r'
%matplotlib inline

## Image Correction
Here is a simple image correction based on our discussions from Chapter 9.

In [ ]:
hdulist = fits.open('data/aa_aql0007_raw.fits')
image_data = hdulist[0].data
image_hdr = hdulist[0].header
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()

## Image Combination
The first step is to combine our calibration frames to make a master calibration. I have 10 biases named `bias001.fits` through `bias010.fits`. Let's use a median combine. We will create a list of bias frames, and give the combined frame the same header as the first frame.

In [ ]:
biaslist = list()
for i in np.arange(10) + 1:
    #Append each bias frame into our list
    filename = 'data/bias{:03d}.fits'.format(i)
    print(filename)
    biaslist.append(fits.open(filename)[0].data)

#Create a 3-d data cube
data_cube = np.array(biaslist)
print(data_cube.shape)

#Now median the images together along the first axis
combined_data = np.median(data_cube,axis=0)
print(combined_data.shape)

#Take image header from first bias
combined_hdr = fits.open('data/bias001.fits')[0].header
#Add a keyword, so we know this is a combined frame
combined_hdr['NCOMBINE'] = 10
fits.writeto('data/Zero.fits', combined_data, combined_hdr,overwrite=True)

In [ ]:
(imin,imax) = interval.get_limits(combined_data)
plt.imshow(combined_data, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Bias Subtraction
The first step is to remove a bias (zeroes) from our image.

In [ ]:
bias = fits.open('data/Zero.fits')
bias_data = bias[0].data
step1 = image_data - bias_data
print(image_data[61,60])
print(step1[61,60])
image_hdr['comment'] = "Subtracted Bias"

In [ ]:
(imin,imax) = interval.get_limits(step1)
plt.imshow(step1, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Dark Subtraction
Step 2 is Dark Subtraction (Note each Calibration image has already had the previous step applied to it).

In [ ]:
dark = fits.open('data/Dark.fits')
dark_hdr = dark[0].header
dark_data = dark[0].data
(imin,imax) = interval.get_limits(dark_data)
plt.imshow(dark_data, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

In [ ]:
step2 = step1 - dark_data
image_hdr['comment'] = "Subtracted Dark"
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Uh-Oh something is wrong?
(imin,imax) = interval.get_limits(step2)
plt.imshow(step2, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

In [ ]:
print(image_hdr['exptime'])
print(dark_hdr['exptime'])

In [ ]:
#We also need to know the dark current.
dark_current = np.median(dark_data/(dark_hdr['exptime']))
print(dark_current)
#Save Dark Current in header
image_hdr['DARKCUR'] = (dark_current,"Dark Current ADU per second")

#Need to correct scale the dark to the exposure time of the science frame
step2 = step1 - (dark_data * (image_hdr['exptime']/dark_hdr['exptime']))
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Much Better
(imin,imax) = interval.get_limits(step2)
plt.imshow(step2, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Flat Fielding
Step 3 is to Flat Field the image. This is a normalization so we are dividing instead of subtracting. `FlatV.fits` is a combined Flat taken in the V filter. It has had both the Zero and Dark removed already.

In [ ]:
flat = fits.open('data/FlatV.fits')
flat_hdr = flat[0].header
flat_data = flat[0].data
(imin,imax) = interval.get_limits(flat_data)
plt.imshow(flat_data, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

In [ ]:
print(np.median(flat_data)) #Is the median level of our Flat
step3 = step2 / flat_data
image_hdr['comment'] = "Divided Flat"
print(image_data[61,60])
print(step1[61,60])
print(step2[61,60]) #Much Better
print(step3[61,60]) #Hmmm this is a little strange
(imin,imax) = interval.get_limits(step3)
plt.imshow(step3, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Save your results
You can save your results by giving the function fits.writeto() your data and an optional header.

In [ ]:
fits.writeto('data/out.fits', step3, image_hdr,overwrite=True)

## Lab 12: Now it is your turn
Please answer the following questions, then print them off and turn them in. You don't need to print the whole notebook. Only print the pages starting from here.

Name:

**Q1: In /data there is a formally reduced image called `aa_aql0007.fits`. Compare it to your image `out.fits`. How good a job did we do?**
* Open both images in ds9
* Print the header of out.fits
* Print fits info about both images

**Q2: Try the following:**
* Plot both images
* Print the min, max, and median of both images 
* Divide both images and plot the result

**Q3: Divide both images. Print the min, max, and median of the resulting image. Plot the resulting image.**

**Q4: Clearly we forgot something in Step 3. Redo Step 3, so that it gives the correct count level for the final image. Print the min, max, and median of the corrected image. Plot the corrected image. Hint: Chapter 9 will give you guidance on this.**